# Imports

In [10]:
import pandas as pd
import h5py
import numpy as np
import tensorflow
from tensorflow import keras

In [8]:
#Lanzar desde la carpeta LightX3ECGPrivate
import sys
sys.path.append('./automatic-ecg-diagnosis/')
# sys.path.append('./source/')
import predict

AttributeError: module 'tensorflow.compat.v2' has no attribute '__internal__'

# Commands

In [1]:
!git clone https://github.com/antonior92/automatic-ecg-diagnosis.git

Clonando en 'automatic-ecg-diagnosis'...
remote: Enumerating objects: 170, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (26/26), done.
remote: Total 170 (delta 35), reused 25 (delta 24), pack-reused 120
Recibiendo objetos: 100% (170/170), 811.98 KiB | 5.27 MiB/s, listo.
Resolviendo deltas: 100% (79/79), listo.


In [12]:
%cd automatic-ecg-diagnosis

/scratch/tfg_luis_2223/automatic-ecg-diagnosis-private/automatic-ecg-diagnosis


In [13]:
!ls

data	      generate_figures_and_tables.py  predict.py	train.py
datasets.py   LICENSE			      __pycache__
data.zip      model.py			      README.md
dnn_predicts  outputs			      requirements.txt


In [11]:
!wget https://www.dropbox.com/s/p3vd3plcbu9sf1o/data.zip?dl=0 -O data.zip

--2023-05-02 18:21:58--  https://www.dropbox.com/s/p3vd3plcbu9sf1o/data.zip?dl=0
Resolviendo www.dropbox.com (www.dropbox.com)... 162.125.68.18, 2620:100:6024:18::a27d:4412
Conectando con www.dropbox.com (www.dropbox.com)[162.125.68.18]:443... conectado.
Petición HTTP enviada, esperando respuesta... 302 Found
Localización: /s/raw/p3vd3plcbu9sf1o/data.zip [siguiendo]
--2023-05-02 18:21:58--  https://www.dropbox.com/s/raw/p3vd3plcbu9sf1o/data.zip
Reutilizando la conexión con www.dropbox.com:443.
Petición HTTP enviada, esperando respuesta... 302 Found
Localización: https://uc776bd503b4554baf42e8e1444b.dl.dropboxusercontent.com/cd/0/inline/B7Qub56Dm4nFxk8XRoEMlBkzsCI3PaHncyWZKodICaR55uM9eMuYuoUZnH-9USPaOcJJ6qJM3mylVU6u86B3BR2k6jWUil7g29fotfKh0d6Kir86bKWE2Qd4ddmw2mvBrzpm5X-3y240SOomVSv8E0sCPCoBATC3cVbe2ZJF0MvPxg/file# [siguiendo]
--2023-05-02 18:21:59--  https://uc776bd503b4554baf42e8e1444b.dl.dropboxusercontent.com/cd/0/inline/B7Qub56Dm4nFxk8XRoEMlBkzsCI3PaHncyWZKodICaR55uM9eMuYuoUZnH-9USP

In [12]:
!unzip data.zip

Archive:  data.zip
  inflating: data/ecg_tracings.hdf5  
replace data/README.md? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


In [8]:
!pip3 show tensorflow

Name: tensorflow
Version: 1.14.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/tfg_luis_2223/.local/lib/python3.7/site-packages
Requires: google-pasta, astor, gast, keras-preprocessing, six, protobuf, wheel, keras-applications, absl-py, tensorboard, wrapt, grpcio, numpy, termcolor, tensorflow-estimator
Required-by: 


In [5]:
!pip install -U pip

    100% |████████████████████████████████| 1.5MB 389kB/s eta 0:00:01


In [7]:
!python -v

# installing zipimport hook
import zipimport # builtin
# installed zipimport hook
# /usr/lib/python2.7/site.pyc matches /usr/lib/python2.7/site.py
import site # precompiled from /usr/lib/python2.7/site.pyc
# /usr/lib/python2.7/os.pyc matches /usr/lib/python2.7/os.py
import os # precompiled from /usr/lib/python2.7/os.pyc
import errno # builtin
import posix # builtin
# /usr/lib/python2.7/posixpath.pyc matches /usr/lib/python2.7/posixpath.py
import posixpath # precompiled from /usr/lib/python2.7/posixpath.pyc
# /usr/lib/python2.7/stat.pyc matches /usr/lib/python2.7/stat.py
import stat # precompiled from /usr/lib/python2.7/stat.pyc
# /usr/lib/python2.7/genericpath.pyc matches /usr/lib/python2.7/genericpath.py
import genericpath # precompiled from /usr/lib/python2.7/genericpath.pyc
# /usr/lib/python2.7/warnings.pyc matches /usr/lib/python2.7/warnings.py
import warnings # precompiled from /usr/lib/python2.7/warnings.pyc
# /usr/lib/python2.7/linecache.pyc matches /usr/lib/python2.7/linecache.

In [6]:
!pip3 install tensorflow==2.2

  Could not find a version that satisfies the requirement tensorflow==2.2 (from versions: 1.13.1, 1.13.2, 1.14.0)
No matching distribution found for tensorflow==2.2


# Code

In [18]:
args= "data/ecg_tracings.hdf5"
with h5py.File(args, "r") as f:
    x = np.array(f['tracings'])
    data = f['tracings']

np.size(x[1])
x[1,:]

array([[ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        , ...,  0.        ,
         0.        ,  0.        ],
       ...,
       [-0.43425145,  0.00380549,  0.43805694, ...,  1.20552059,
         1.66197859, -0.76743238],
       [-0.40663559,  0.01588737,  0.42252296, ...,  1.20201614,
         1.65950262, -0.76712383],
       [-0.38198727,  0.02548188,  0.40746919, ...,  1.20299696,
         1.65952942, -0.75982732]])

In [19]:
f['tracings']

ValueError: Not a location (invalid object ID)

In [31]:
!python3 train.py -h

/home/tfg_luis_2223/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/tfg_luis_2223/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/tfg_luis_2223/.local/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/tfg_luis_2223/.local/lib/python3.7/site-packages/tensorflow/pyth

In [3]:
PFOLDER="./dnn_predicts"
MFOLDER="./model"
DFOLDER="../data" #Nos hemos cambiado al directorio de automatic-ecg-diagnosis

!python predict.py $DFOLDER/ecg_tracings.hdf5 $MFOLDER/model.hdf5 --output_file $PFOLDER/model_mad.npy

Traceback (most recent call last):
  File "predict.py", line 5, in <module>
    from tensorflow.keras.models import load_model
ImportError: No module named tensorflow.keras.models


In [7]:
!python predict.py -h

Traceback (most recent call last):
  File "predict.py", line 5, in <module>
    from tensorflow import keras
ImportError: No module named tensorflow
